In [1]:
import pandas as pd
import torch
from transformers import BertForSequenceClassification, BertTokenizer
from sklearn.metrics import classification_report

In [2]:
# Define the mapping from EPRL tags to integers
eprl_map = {
    'Action (How)': 0,
    'Action (What)': 1,
    'Action (Where)': 2,
    'Decision (What)': 3,
    'Trigger (How)': 4,
    'Trigger (What)': 5,
    'Trigger (Where)': 6,
    'Verification (How)': 7,
    'Verification (What)': 8,
    'Verification (Where)': 9,
    'Waiting (How)': 10,
    'Waiting (What)': 11,
    'Waiting (Where)': 12
}

In [14]:
# Load the data
data = pd.read_csv('C:/Users/Custom/Downloads/maybe.csv')
data

,FILE,TEXT,EPRL
0,HAL_2_3.pdf,As Required,Trigger (What)
1,HAL_2_3.pdf,As required,Trigger (What)
2,HAL_2_3.pdf,As required,Trigger (What)
3,HAL_2_3.pdf,As required,Trigger (What)
4,HAL_2_3.pdf,As required,Trigger (What)
...,...,...,...
1180,HAL_5_999.pdf,right,Verification (Where)
1181,HAL_5_999.pdf,left,Verification (Where)
1182,HAL_5_999.pdf,bottom of screen,Verification (Where)
1183,HAL_5_999.pdf,bottom,Verification (Where)


In [15]:
# Map the EPRL tags to integers
data['EPRL'] = data['EPRL'].map(eprl_map)

In [16]:
# Load the BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(eprl_map))
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

C:\Users\Custom\anaconda3\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Custom\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [17]:
# Define a function to preprocess the text data
def preprocess(text):
    encoded_text = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=64,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )
    return encoded_text

In [18]:
# Preprocess the text data and convert it to a tensor
inputs = data['TEXT'].apply(preprocess)
input_ids = torch.cat([i['input_ids'] for i in inputs])
attention_masks = torch.cat([i['attention_mask'] for i in inputs])
labels = torch.tensor(data['EPRL'].values)

In [19]:
# Define a DataLoader to feed the input to the model
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(input_ids, attention_masks, labels)
dataloader = DataLoader(dataset, batch_size=32)

In [20]:
# Define the training and evaluation loops
def train(model, dataloader, optimizer):
    model.train()
    for batch in dataloader:
        optimizer.zero_grad()
        outputs = model(input_ids=batch[0], attention_mask=batch[1], labels=batch[2])
        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()

In [21]:
def evaluate(model, dataloader):
    model.eval()
    predictions = []
    true_labels = []
    for batch in dataloader:
        with torch.no_grad():
            outputs = model(input_ids=batch[0], attention_mask=batch[1])
            logits = outputs.logits
            predictions += logits.argmax(-1).cpu().numpy().tolist()
            true_labels += batch[2].cpu().numpy().tolist()
    report = classification_report(true_labels, predictions, target_names=list(eprl_map.keys()))
    return report

In [22]:
# Define the training parameters
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
num_epochs = 10

In [ ]:
# Train the model
for epoch in range(num_epochs):
    train(model, dataloader, optimizer)
    report = evaluate(model, dataloader)
    print(f'Epoch {epoch + 1}:\n{report}')
    
# Make predictions on the test set
model.eval()
predictions = []
for batch in dataloader:
    with torch.no_grad():
        outputs = model(input_ids=batch[0], attention_mask=batch[1])
        logits = outputs.logits
        predictions += logits.argmax(-1).cpu().numpy().tolist()
        
# Map the predicted integer labels back to EPRL tags
predicted_eprl = pd.Series(predictions).map({v: k for k, v in eprl_map.items()})

# Add the predicted EPRL tags to the original data
data['EPRL_predicted'] = predicted_eprl

In [ ]:
# Save the updated data to a CSV file
data.to_csv('maybe_with_predictions.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

# Define the epoch and accuracy values
epochs = [1, 4, 5, 6, 7, 8, 9, 10]
accuracies = [0.78, 0.86, 0.8, 0.9, 0.91, 0.93, 0.93, 0.94]
weighted_avgs = [0.74, 0.82, 0.76, 0.88, 0.89, 0.92, 0.92, 0.93]
macro_avgs = [0.37, 0.41, 0.43, 0.49, 0.58, 0.68, 0.68, 0.73]

# Create three subplots for the different metrics
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(8, 10))

# Plot the accuracy on the first subplot
ax1.plot(epochs, accuracies)
ax1.set_ylabel('Accuracy')

# Plot the weighted average on the second subplot
ax2.plot(epochs, weighted_avgs)
ax2.set_ylabel('Weighted Average')

# Plot the macro average on the third subplot
ax3.plot(epochs, macro_avgs)
ax3.set_xlabel('Epoch')
ax3.set_ylabel('Macro Average')

# Add a title to the plot
plt.suptitle('Model Performance by Epoch')

# Show the plot
plt.show()
